**Group Name:** \_\_\_\_\_

**Members:** \_\_\_\_\_

## Template of the Train Code
The train code template is shown below:

In [2]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm 

device = "cuda" if torch.cuda.is_available() else "cpu"

# ===============================
# Self-Defined Dataloader
# ===============================
class data_loader(Dataset):
    def __init__(self, data_dir):

        real = os.path.join(data_dir, '0_real')
        fake = os.path.join(data_dir, '1_fake')

        file_names_real = os.listdir(real)
        file_names_fake = os.listdir(fake)

        self.full_filenames_real = [os.path.join(real, f) for f in file_names_real]
        self.full_filenames_fake = [os.path.join(fake, f) for f in file_names_fake]
        self.full_filenames = self.full_filenames_real + self.full_filenames_fake

        self.labels_real = [0 for _ in file_names_real]
        self.labels_fake = [1 for _ in file_names_fake]
        self.labels = self.labels_real + self.labels_fake

        self.transform = transforms.Compose([
            transforms.RandomResizedCrop(size=(64, 64)),
            transforms.ToTensor(),
        ])
        
    def __len__(self):
        return len(self.full_filenames)

    def __getitem__(self, idx):
        image = Image.open(self.full_filenames[idx]).convert("RGB")
        image = self.transform(image)
        label = self.labels[idx]
        return image, label


# ===============================
# Neural NetWork
# ===============================
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))   
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = self.avgpool(x)         
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# ===============================
# Train-Validate
# ===============================
def main():
    data_root = "data"
    batch_size = 32
    epochs = 10
    lr = 1e-4

    train_dataset = data_loader(os.path.join(data_root, "train"))
    val_dataset = data_loader(os.path.join(data_root, "val"))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # ---- Train ----
        model.train()
        total_loss, total_correct, total = 0, 0, 0
        loop = tqdm(train_loader, total=len(train_loader), desc=f"Epoch [{epoch+1}/{epochs}]")

        for imgs, labels in loop:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * imgs.size(0)
            total_correct += (outputs.argmax(1) == labels).sum().item()
            total += imgs.size(0)

        train_loss = total_loss / total
        train_acc = total_correct / total

        # ---- Validate ----
        model.eval()
        val_correct, val_total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                preds = outputs.argmax(1)
                val_correct += (preds == labels).sum().item()
                val_total += imgs.size(0)
        val_acc = val_correct / val_total

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Save Model
    torch.save(model.state_dict(), "model.pth")
    print("Model saved")

if __name__ == "__main__":
    main()


Epoch [1/10]: 100%|██████████| 1563/1563 [04:47<00:00,  5.43it/s]


Epoch [1/10] Train Loss: 0.5682 | Train Acc: 0.7058 | Val Acc: 0.7352


Epoch [2/10]: 100%|██████████| 1563/1563 [04:49<00:00,  5.39it/s]


Epoch [2/10] Train Loss: 0.5201 | Train Acc: 0.7466 | Val Acc: 0.7545


Epoch [3/10]: 100%|██████████| 1563/1563 [04:50<00:00,  5.37it/s]


...

Epoch [6/10]: 100%|██████████| 1563/1563 [04:51<00:00,  5.37it/s]


Epoch [6/10] Train Loss: 0.4683 | Train Acc: 0.7794 | Val Acc: 0.7772


Epoch [7/10]: 100%|██████████| 1563/1563 [04:51<00:00,  5.35it/s]


Epoch [7/10] Train Loss: 0.4613 | Train Acc: 0.7851 | Val Acc: 0.7837


Epoch [8/10]: 100%|██████████| 1563/1563 [04:52<00:00,  5.35it/s]
